# Project 6: AirBnB data modelling with Neo4j

## Data loading into Neo4j

In [1]:
%env CYPHER=/home/jupyter/neo4j-community-4.1.3/bin/cypher-shell
%env USER=neo4j
%env PW=neo4j

env: CYPHER=/home/jupyter/neo4j-community-4.1.3/bin/cypher-shell
env: USER=neo4j
env: PW=neo4j


In [2]:
CONNECT="$CYPHER -u $USER -p $PW"

In [3]:
!{CONNECT} "SHOW DATABASES"

+------------------------------------------------------------------------------------------------+
| name     | address          | role         | requestedStatus | currentStatus | error | default |
+------------------------------------------------------------------------------------------------+
| "neo4j"  | "localhost:7687" | "standalone" | "online"        | "online"      | ""    | TRUE    |
| "system" | "localhost:7687" | "standalone" | "online"        | "online"      | ""    | FALSE   |
+------------------------------------------------------------------------------------------------+

2 rows available after 2 ms, consumed after another 2 ms


In [4]:
!{CONNECT} "MATCH (n) DETACH DELETE n"
!{CONNECT} "MATCH (n) RETURN n"

0 rows available after 1223 ms, consumed after another 0 ms
Deleted 129444 nodes, Deleted 220183 relationships
+---+
| n |
+---+
+---+

0 rows available after 6 ms, consumed after another 3 ms


In [5]:
!{CONNECT} "CALL db.constraints"

+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
| name                  | description                                                                                   | details                                                                                                                     |
+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
| "constraint_3988c775" | "CONSTRAINT ON ( amenity:Amenity ) ASSERT (amenity.name) IS UNIQUE"                           | "Constraint( id=5, name='constraint_3988c775', type='UNIQUENESS', schema=(:Amenity {name}), ownedIndex=8 )"                 |
| "const

In [6]:
!{CONNECT} "DROP CONSTRAINT constraint_3988c775"
!{CONNECT} "DROP CONSTRAINT constraint_3e37701d"
!{CONNECT} "DROP CONSTRAINT constraint_c0af93de"
!{CONNECT} "DROP CONSTRAINT constraint_c2c3e4d2"

0 rows available after 2 ms, consumed after another 0 ms
Removed 1 constraints
0 rows available after 7 ms, consumed after another 0 ms
Removed 1 constraints
0 rows available after 5 ms, consumed after another 0 ms
Removed 1 constraints
0 rows available after 3 ms, consumed after another 0 ms
Removed 1 constraints


In [7]:
!{CONNECT} "CALL db.indexes"

+-------------------------------------------------------------------------------------------------------------------------------------------------+
| id | name             | state    | populationPercent | uniqueness  | type    | entityType | labelsOrTypes | properties     | provider           |
+-------------------------------------------------------------------------------------------------------------------------------------------------+
| 3  | "index_cc55e8e7" | "ONLINE" | 100.0             | "NONUNIQUE" | "BTREE" | "NODE"     | ["Listing"]   | ["listing_id"] | "native-btree-1.0" |
+-------------------------------------------------------------------------------------------------------------------------------------------------+

1 row available after 14 ms, consumed after another 0 ms


In [8]:
!{CONNECT} "DROP INDEX ON:Listing(listing_id)"
!{CONNECT} "DROP INDEX ON:Amenity(name)"
!{CONNECT} "DROP INDEX ON:Host(host_id)"
!{CONNECT} "DROP INDEX ON:User(user_id)"
!{CONNECT} "DROP INDEX ON:Neighborrhood(neighborhood_id)"

0 rows available after 2 ms, consumed after another 0 ms
Removed 1 indexes
Unable to drop index on (:Amenity {name}). There is no such index.
Unable to drop index on (:Host {host_id}). There is no such index.
Unable to drop index on (:User {user_id}). There is no such index.
Unable to drop index on (:Neighborrhood {neighborhood_id}). There is no such index.


In [9]:
#!{CONNECT} "DROP INDEX ON:Listing(listing_id)"
!cat /home/jupyter/airbnb/load_data.cypher | {CONNECT} --format plain


COUNT(l)
5835
COUNT(a)
42
COUNT(n)
41
COUNT(h)
4633
COUNT(u)
55917
COUNT(r)
62976


## Node label counts

In [10]:
!{CONNECT} "MATCH (n) RETURN COUNT(n)"

+----------+
| COUNT(n) |
+----------+
| 129444   |
+----------+

1 row available after 6 ms, consumed after another 0 ms


In [11]:
!{CONNECT} "MATCH (n:Neighborhood) RETURN COUNT(n)"

+----------+
| COUNT(n) |
+----------+
| 41       |
+----------+

1 row available after 8 ms, consumed after another 1 ms


In [12]:
!{CONNECT} "MATCH (l:Listing) RETURN COUNT(l)"

+----------+
| COUNT(l) |
+----------+
| 5835     |
+----------+

1 row available after 6 ms, consumed after another 0 ms


In [13]:
!{CONNECT} "MATCH (r:Review) RETURN COUNT(r)"

+----------+
| COUNT(r) |
+----------+
| 62976    |
+----------+

1 row available after 1 ms, consumed after another 1 ms


In [14]:
!{CONNECT} "MATCH (u:User) RETURN COUNT(u)"

+----------+
| COUNT(u) |
+----------+
| 55917    |
+----------+

1 row available after 1 ms, consumed after another 0 ms


In [15]:
!{CONNECT} "MATCH (h:Host) RETURN COUNT(h)"

+----------+
| COUNT(h) |
+----------+
| 4633     |
+----------+

1 row available after 8 ms, consumed after another 0 ms


In [16]:
!{CONNECT} "MATCH (a:Amenity) RETURN COUNT(a)"

+----------+
| COUNT(a) |
+----------+
| 42       |
+----------+

1 row available after 6 ms, consumed after another 0 ms


In [17]:
!{CONNECT} "MATCH (c:City) RETURN COUNT(c)"

+----------+
| COUNT(c) |
+----------+
| 0        |
+----------+

1 row available after 9 ms, consumed after another 0 ms


In [18]:
!{CONNECT} "MATCH (is:IN_STATE) RETURN COUNT(is)"

+-----------+
| COUNT(is) |
+-----------+
| 0         |
+-----------+

1 row available after 8 ms, consumed after another 0 ms


In [19]:
!{CONNECT} "MATCH (c:Country) RETURN COUNT(c)"

+----------+
| COUNT(c) |
+----------+
| 0        |
+----------+

1 row available after 7 ms, consumed after another 0 ms


## Cypher Queries

Q1. How many hosts are located in "Austin, Texas, United States"?

In [20]:
!{CONNECT} "MATCH (n:Host {{location: 'Austin, Texas, United States'}}) RETURN COUNT(n)"

+----------+
| COUNT(n) |
+----------+
| 3774     |
+----------+

1 row available after 11 ms, consumed after another 5 ms


Q2. Which listings does host_id = "4641823" have? Return the listing name, property_type,
price, and availability_365 sorted by price. Limit the results to 10.

In [21]:
!{CONNECT} "MATCH (h:Host {{host_id: '4641823'}})-[n:HOSTS]-(l:Listing) RETURN l.name, l.property_type, l.price, l.availability_365 ORDER BY l.price LIMIT 10"

+----------------------------------------------------------------------------------------+
| l.name                                | l.property_type | l.price | l.availability_365 |
+----------------------------------------------------------------------------------------+
| "1BR Convenient Austin Condo "        | "Apartment"     | 93.0    | 354                |
| "1BR Inviting Downtown Condo, Pool"   | "Apartment"     | 99.0    | 335                |
| "2BR/1.5BA Charming House Off SoCo"   | "House"         | 110.0   | 357                |
| "2BR Prime East-Side Downtown"        | "House"         | 121.0   | 341                |
| "1BR SoCo Treehouse Studio"           | "House"         | 129.0   | 327                |
| "1BR/1.5BA East 6th, Colorful 2Story" | "Apartment"     | 134.0   | 344                |
| "3BR Prestigious Home Travis Heights" | "House"         | 138.0   | 0                  |
| "1BR/1.5BA Perfectly Located Casita"  | "House"         | 140.0   | 351                |

Q3. Which users wrote a review for listing_id = "5293632"? Return the user’s id and name
sorted alphabetically by name. Limit the results to 10.

In [22]:
!{CONNECT} "MATCH (l:Listing {{listing_id: '5293632'}})<-[:REVIEWS]-(r:Review)<-[:WROTE]-(u:User) RETURN u.user_id, u.name ORDER BY u.name LIMIT 10"

+--------------------------------+
| u.user_id  | u.name            |
+--------------------------------+
| "18286390" | "Annie"           |
| "30193020" | "Carole"          |
| "16497582" | "Cory"            |
| "35022795" | "Craig And Trina" |
| "13281665" | "Dianne"          |
| "29601600" | "Hannah"          |
| "11940539" | "Jacob"           |
| "3213433"  | "Jessie"          |
| "41722221" | "Johannes"        |
| "28480571" | "Ju-Ju"           |
+--------------------------------+

10 rows available after 26 ms, consumed after another 1 ms


Q4. Which users wrote a review for any listing which has the amenities "Washer" and "Dryer"?
Return the user’s id and name sorted alphabetically by name. Limit the results to 10.

In [23]:
!{CONNECT} "MATCH (b:Amenity{{name:'Washer'}})-[:HAS]-(l:Listing)-[:HAS]-(a:Amenity{{name:'Dryer'}}) MATCH (l)<-[:REVIEWS]-(r:Review)<-[:WROTE]-(u:User) RETURN u.user_id, u.name ORDER BY u.name LIMIT 10"


+-------------------------------------+
| u.user_id  | u.name                 |
+-------------------------------------+
| "6524431"  | "'Ley"                 |
| "8026901"  | "(We Are) Bonnie & Ky" |
| "15315643" | "(email hidden)"       |
| "5317970"  | "(email hidden)"       |
| "7802424"  | "(email hidden)"       |
| "5164376"  | "(email hidden)"       |
| "10469354" | "(email hidden)"       |
| "12694638" | "(email hidden)"       |
| "7802424"  | "(email hidden)"       |
| "12694638" | "(email hidden)"       |
+-------------------------------------+

10 rows available after 90 ms, consumed after another 167 ms


Q5. Which listings have 3 bedrooms and are located in the Clarksville neighborhood? Return
the listing name, property_type, price, and availability_365 sorted by price. Limit the results
to 5.

In [24]:
!{CONNECT} "MATCH (l:Listing {{bedrooms: 3}})-[:IN_NEIGHBORHOOD]->(n:Neighborhood{{name: 'Clarksville'}}) RETURN l.name, l.property_type, l.price, l.availability_365 ORDER BY l.price LIMIT 5"

+----------------------------------------------------------------------------------------+
| l.name                                | l.property_type | l.price | l.availability_365 |
+----------------------------------------------------------------------------------------+
| "3BR/2.5BA Exquisite Townhouse"       | "House"         | 222.0   | 358                |
| "3BR/2.5BA Tarrytown Duplex, Austin!" | "House"         | 249.0   | 336                |
| "Austin downtown hideaway"            | "House"         | 249.0   | 364                |
| "3BD Luxury Cottage by Lake Austin"   | "House"         | 290.0   | 309                |
| "Entire Adorable Downtown House"      | "House"         | 295.0   | 309                |
+----------------------------------------------------------------------------------------+

5 rows available after 27 ms, consumed after another 1 ms


Q6. Which amenities are the most common? Return the name of the amenity and its frequency.
Sort the results by count in descending order. Limit the results to 5.

In [25]:
!{CONNECT} "MATCH (l:Listing)-[:HAS]->(a:Amenity) RETURN a.name, count(a) as frequency ORDER BY frequency DESC LIMIT 5" 

+----------------------------------------+
| a.name                     | frequency |
+----------------------------------------+
| "Air Conditioning"         | 5615      |
| "Wireless Internet"        | 5479      |
| "Heating"                  | 5440      |
| "Kitchen"                  | 5400      |
| "Free Parking on Premises" | 5123      |
+----------------------------------------+

5 rows available after 14 ms, consumed after another 206 ms


Q7. Which neighborhoods have the highest number of listings? Return the neighborhood’s
name and zip code (neighborhood_id) and number of listings they have sorted by number
of listings in descending order. Limit the results to 5.

In [26]:
!{CONNECT} "MATCH (l:Listing)-[:IN_NEIGHBORHOOD]->(n:Neighborhood) RETURN n.name, n.neighborhood_id, count(l) as numListings ORDER BY numListings DESC LIMIT 5" 

+----------------------------------------------------+
| n.name           | n.neighborhood_id | numListings |
+----------------------------------------------------+
| NULL             | "78704"           | 1601        |
| NULL             | "78702"           | 797         |
| "Clarksville"    | "78703"           | 419         |
| "East Riverside" | "78741"           | 414         |
| NULL             | "78745"           | 328         |
+----------------------------------------------------+

5 rows available after 15 ms, consumed after another 29 ms
